This notebook provides a short sample on how to train a s2s benchmark model. It uses `pytorch lightning` module, and uses MLP as an example.

The complete training script can be found in the root directory of the repository `train.py`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import yaml
import torch
import lightning.pytorch as pl
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint
pl.seed_everything(42)

import sys
sys.path.append('..')

from chaosbench import dataset, config, utils, criterion
from chaosbench.models import mlp, model


[rank: 0] Global seed set to 42


In [3]:
# Load config filepath which consists of all the definition needed to fit/eval a model

model_config_filepath = '../chaosbench/configs/mlp_s2s.yaml'
with open(model_config_filepath, 'r') as config_filepath:
    hyperparams = yaml.load(config_filepath, Loader=yaml.FullLoader)

model_args = hyperparams['model_args']
data_args = hyperparams['data_args']


In [4]:
# This is how the hyperparameters are structured: 
# `model_args` for model specification
# `data_args` for data definition

hyperparams


{'model_args': {'model_name': 'mlp_s2s',
  'input_size': 60,
  'hidden_sizes': [768, 640, 512, 640, 640],
  'output_size': 60,
  'learning_rate': 0.001,
  'num_workers': 0,
  'epochs': 300},
 'data_args': {'batch_size': 32,
  'train_years': [2016, 2017, 2018, 2019, 2020, 2021],
  'val_years': [2022]}}

In [4]:
# Initialize model
# By passing the necessary hyperparameters (model + dataset)

baseline = model.S2SBenchmarkModel(model_args=model_args, data_args=data_args)
baseline.setup()


In [5]:
# Setup trainer
# Including tensorboard logger and checkpoint callback (eg. saving top-1 based on lowest validation error)

tb_logger = pl_loggers.TensorBoardLogger(save_dir=f'logs/{model_args.model_name}')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')

trainer = pl.Trainer(
    devices=1,
    accelerator='gpu',
    strategy='auto',
    max_epochs=model_args['epochs'],
    logger=tb_logger,
    callbacks=[checkpoint_callback]
 )


/burg/home/jn2808/.conda/envs/bench/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /burg/home/jn2808/.conda/envs/bench/lib/python3.9/si ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# Fit the model
# Checkpoint can be found under `logs/<MODEL_NAME>`

trainer.fit(baseline)
